![pipeline](image.png)


combine context + question -> model -> dự đoán ra 2 vị trí đầu và cuối của câu trả lời trong context 

### 1. Data

In [1]:
qa_dataset = [
    {
        'context': 'My name is AIVN and I am from Vietnam.',
        'question': 'What is my name?',
        'answer': 'AIVN'
    },
    {
        'context': 'I love painting and my favorite artist is Vincent Van Gogh.',
        'question': 'What is my favorite activity?',
        'answer': 'painting'
    },
    {
        'context': 'I am studying computer science at the University of Tokyo.',
        'question': 'What am I studying?',
        'answer': 'computer science'
    },
    {
        'context': 'My favorite book is "To Kill a Mockingbird" by Harper Lee.',
        'question': 'What is my favorite book?', 
        'answer': '"To Kill a Mockingbird"'
    }
]

### 2. Vectorization

In [2]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator


tokenier = get_tokenizer("basic_english")

def yield_tokens(examples: list):
    for item in examples:
        yield tokenier(item["context"] + " <sep> " + item["question"])


vocab = build_vocab_from_iterator(
    iterator=yield_tokens(qa_dataset), 
    specials=["<unk>", "<pad>", "<bos>", "<eos>", "<sep>"]
)
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

/home/ducpham/anaconda3/lib/python3.11/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/ducpham/anaconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/ducpham/anaconda3/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHT

{'vincent': 40,
 'vietnam': 39,
 'university': 37,
 'to': 35,
 'the': 34,
 'painting': 32,
 'of': 31,
 'mockingbird': 30,
 'am': 12,
 'what': 11,
 '<sep>': 4,
 '<bos>': 2,
 'science': 33,
 '?': 8,
 'my': 6,
 'is': 5,
 'at': 21,
 'gogh': 25,
 'love': 29,
 'lee': 28,
 '.': 7,
 '<eos>': 3,
 '<pad>': 1,
 'computer': 23,
 'artist': 20,
 'favorite': 9,
 'harper': 26,
 '<unk>': 0,
 'and': 13,
 'studying': 16,
 'i': 10,
 'aivn': 19,
 'van': 38,
 'book': 14,
 'tokyo': 36,
 'name': 15,
 'kill': 27,
 'by': 22,
 'a': 17,
 'from': 24,
 'activity': 18}

In [3]:
PAD_IDX = vocab["<pad>"]

def pad_and_truncate(input_ids: list[int], max_seq_len: int):
    if len(input_ids) > max_seq_len:
        input_ids = input_ids[:max_seq_len]
    else:
        input_ids = input_ids + [PAD_IDX] * (max_seq_len - len(input_ids))
    return input_ids


MAX_SEQ_LEN = 22
text = "I love AIVN"
tokenized_text = tokenier(text)
tokens = [vocab[token] for token in tokenized_text]
print(tokens)
tokens = pad_and_truncate(input_ids=tokens, max_seq_len=MAX_SEQ_LEN)
print(tokens)

[10, 29, 19]
[10, 29, 19, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [4]:
import torch


def vectorize(question: str, context: str, answer: str):
    
    input_text = question + " <sep> " + context
    input_ids = [vocab[token] for token in tokenier(input_text)]
    input_ids = pad_and_truncate(input_ids=input_ids, max_seq_len=MAX_SEQ_LEN)

    answer_ids = [vocab[token] for token in tokenier(answer)]
    st_pos = input_ids.index(answer_ids[0])
    end_pos = st_pos + len(answer_ids) - 1

    input_ids = torch.tensor(input_ids, dtype=torch.long)
    st_pos = torch.tensor(st_pos, dtype=torch.long)
    end_pos = torch.tensor(end_pos, dtype=torch.long)
    return input_ids, st_pos, end_pos

input_ids, st_pos, end_pos = vectorize(
    question=qa_dataset[0]['question'],
    context=qa_dataset[0]['context'], 
    answer=qa_dataset[0]['answer']
)
print(input_ids)
print(st_pos)
print(end_pos)

tensor([11,  5,  6, 15,  8,  4,  6, 15,  5, 19, 13, 10, 12, 24, 39,  7,  1,  1,
         1,  1,  1,  1])
tensor(9)
tensor(9)


In [5]:
id2token = {id: label for label, id in vocab.get_stoi().items()}
for token in input_ids.numpy():
    print(id2token[token], end= ' ')

what is my name ? <sep> my name is aivn and i am from vietnam . <pad> <pad> <pad> <pad> <pad> <pad> 

### 3. Create dataset

In [6]:

from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, data: list[dict]):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        question_text = item['question']
        context_text = item['context']
        answer_text = item['answer']

        input_ids, st_pos, end_pos = vectorize(
            question=question_text,
            context=context_text,
            answer=answer_text
        )
        return input_ids, st_pos, end_pos


In [7]:
from torch.utils.data import DataLoader


train_dataset = QADataset(data=qa_dataset)
train_dataloader = DataLoader(dataset=train_dataset, 
                              batch_size=1, 
                              shuffle=True)

for batch in train_dataloader:
    input_ids, st_pos, end_pos = batch
    print(input_ids)
    print(st_pos)
    print(end_pos)
    print("=" * 100)

tensor([[11,  5,  6,  9, 14,  8,  4,  6,  9, 14,  5, 35, 27, 17, 30, 22, 26, 28,
          7,  1,  1,  1]])
tensor([11])
tensor([14])
tensor([[11,  5,  6,  9, 18,  8,  4, 10, 29, 32, 13,  6,  9, 20,  5, 40, 38, 25,
          7,  1,  1,  1]])
tensor([9])
tensor([9])
tensor([[11,  5,  6, 15,  8,  4,  6, 15,  5, 19, 13, 10, 12, 24, 39,  7,  1,  1,
          1,  1,  1,  1]])
tensor([9])
tensor([9])
tensor([[11, 12, 10, 16,  8,  4, 10, 12, 16, 23, 33, 21, 34, 37, 31, 36,  7,  1,
          1,  1,  1,  1]])
tensor([9])
tensor([10])


### 4. Model

In [8]:
import torch.nn as nn 

class QAModel(nn.Module):
    def __init__(self, 
                 vocab_size: int, 
                 embed_dim: int, 
                 hidden_size: int, 
                 n_layers: int, 
                 n_classes: int):
        
        super().__init__()
        self.embed_model = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embed_dim
        )

        self.model = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_size, 
            batch_first=True, 
            bidirectional=True,
            num_layers=n_layers,
        )

        self.st_linear = nn.Linear(in_features=hidden_size*2,
                                    out_features=n_classes)

        self.end_linear = nn.Linear(in_features=hidden_size*2, 
                                    out_features=n_classes)
        
    def forward(self, input_text): 
        # input_text: [N, max_seq_len]
        input_embed = self.embed_model(input_text) # [N, max_seq_len, embed_dim]

        lstm_output, (hidden_lstm, cell_lstm) = self.model(input_embed)
        # lstm_output: [N, max_seq_len, hidden_size * 2]
        # hidden_lstm: [num_layers, N, hidden_size]
        # cell_lstm: [num_layers, N, hidden_size]
        st_logits = self.st_linear(lstm_output).squeeze(-1) # [N, max_seq_len, 1]
        end_logits = self.end_linear(lstm_output).squeeze(-1) # [N, max_seq_len, 1]
        
        return st_logits, end_logits
    


In [9]:
EMBEDDING_DIM = 64
HIDDEN_SIZE = 128
VOCAB_SIZE = len(vocab)
N_LAYERS  = 2
N_CLASSES = 1

model =QAModel(vocab_size=VOCAB_SIZE, 
               embed_dim=EMBEDDING_DIM,
               hidden_size=HIDDEN_SIZE,
               n_layers=N_LAYERS,
               n_classes=N_CLASSES)

input_text =torch.randint(low=0, high=1, size=(1, MAX_SEQ_LEN))

model.eval()
with torch.no_grad():
    st_logits, end_logits = model(input_text)
    print(st_logits.shape)
    print(end_logits.shape)


torch.Size([1, 22])
torch.Size([1, 22])


### 5. Training

In [10]:
LR = 1e-3
EPOCHS = 20
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

criterion = nn.CrossEntropyLoss()

model.train()
for _ in range(EPOCHS):
    for idx, (input_ids, st_pos, end_pos) in enumerate(train_dataloader):
        optimizer.zero_grad()

        st_pos_logits, end_pos_logits = model(input_ids)
        st_loss = criterion(st_pos_logits, st_pos)
        end_loss = criterion(end_pos_logits, end_pos)
        loss = (st_loss + end_loss) / 2
        
        loss.backward()
        optimizer.step()

        print(loss.item())
        
        

3.089033603668213
3.080734968185425
3.047706127166748
3.058602809906006
2.968336582183838
2.9694743156433105
2.8675239086151123
2.7737717628479004
2.772329807281494
2.646535873413086
2.565298318862915
2.569988250732422
2.3456497192382812
2.162170886993408
2.086494207382202
2.0802884101867676
1.6717901229858398
1.6423993110656738
1.5897448062896729
1.3732277154922485
1.103492259979248
1.2283656597137451
1.1151273250579834
0.8588740825653076
0.6452548503875732
1.058375597000122
0.7677636742591858
0.46890974044799805
0.40237319469451904
0.8452520370483398
0.36408132314682007
0.391229510307312
0.2423102706670761
0.2340986430644989
0.695274829864502
0.22322937846183777
0.12938430905342102
0.07479516416788101
0.14029961824417114
0.3061040937900543
0.21380019187927246
0.23403924703598022
0.03682659566402435
0.041764240711927414
0.04405006766319275
0.13051454722881317
0.02148337848484516
0.1496201604604721
0.11250707507133484
0.01291143149137497
0.015408702194690704
0.03128960356116295
0.02618

In [16]:
model.eval()
with torch.no_grad():
    sample = qa_dataset[3]
    context, question, answer = sample.values()
    input_ids, st_pos, end_pos = vectorize(question=question, 
                                           context=context, 
                                           answer=answer)
    input_ids = input_ids.unsqueeze(0) # add batch dimention

    st_logits, end_logits = model(input_ids)

    offset = len(tokenier(question)) + 1
    st_pos = torch.argmax(st_logits, dim=1).numpy()[0]
    end_pos = torch.argmax(end_logits, dim=1).numpy()[0]
    
    st_pos -= offset
    end_pos -= offset

    st_pos = max(st_pos, 0)
    end_pos = min(end_pos, len(tokenier(context)) -1) 

    if end_pos >= st_pos:
        context_tokens = tokenier(context)
        predicted_answer_tokens = context_tokens[st_pos: end_pos +1]
        predicted_answer = " ".join(predicted_answer_tokens)
    else:
        predicted_answer = " "
    
    print(f"Context: {context}")
    print(f"Question: {question}")
    print(f"Prediction: {predicted_answer}")
    

# LSTM QA extractive approach 

Context: My favorite book is "To Kill a Mockingbird" by Harper Lee.
Question: What is my favorite book?
Prediction: to kill a mockingbird
